### Imports

In [1]:
from ultralytics import YOLO
import cv2
import numpy as np

import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt

### Loading Models

In [2]:
model = YOLO("yolov8n.yaml") # blank with pretrained weights
#model = YOLO("D:/Uni Stuff/IP/Coding/Repository/Blimp-Vision-System/runs/detect/3 64/weights/best.pt")  # pretrained model


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.Conv                  [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.Conv                  [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.C2f                   [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.Conv                  [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.C2f                   [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.Conv                  [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.C2f                   [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.Conv                  [128

### Training Models

In [3]:
model.train(data="blimps.yaml", epochs=5, batch=64, imgsz=640, name="5 64", exist_ok=True, verbose=True)

Ultralytics YOLOv8.0.70  Python-3.11.3 torch-2.0.0+cpu CPU
yolo\engine\trainer: task=detect, mode=train, model=yolov8n.yaml, data=blimps.yaml, epochs=5, patience=50, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=5 64, exist_ok=True, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=None, workspace=4, n

### Predicting

In [8]:
results = model.predict(source="D:/Uni Stuff/IP/Data/YOLO Data/test/images", save_conf=True, line_thickness=1, max_det=1)



    WARNING  stream/video/webcam/dir predict source will accumulate results in RAM unless `stream=True` is passed,
    causing potential out-of-memory errors for large sources or long-running streams/videos.

    Usage:
        results = model(source=..., stream=True)  # generator of Results objects
        for r in results:
            boxes = r.boxes  # Boxes object for bbox outputs
            masks = r.masks  # Masks object for segment masks outputs
            probs = r.probs  # Class probabilities for classification outputs

image 1/2627 D:\Uni Stuff\IP\Data\YOLO Data\test\images\background_102_0.png: 384x640 (no detections), 60.0ms
image 2/2627 D:\Uni Stuff\IP\Data\YOLO Data\test\images\background_102_2.png: 384x640 1 blimp, 57.0ms
image 3/2627 D:\Uni Stuff\IP\Data\YOLO Data\test\images\background_103_1.png: 384x640 1 blimp, 60.4ms
image 4/2627 D:\Uni Stuff\IP\Data\YOLO Data\test\images\background_106_0.png: 384x640 (no detections), 57.2ms
image 5/2627 D:\Uni Stuff\IP\Data\YOL

KeyboardInterrupt: 

In [16]:
r = model.predict(source="D:/Uni Stuff/IP/Data/YOLO Data/test/images/image_14521.png", save_conf=True, line_thickness=1, max_det=1)

cv2.imshow("Predicted Image", r[0].plot())

cv2.waitKey(0)
cv2.destroyAllWindows()


image 1/1 D:\Uni Stuff\IP\Data\YOLO Data\test\images\image_14521.png: 384x640 1 blimp, 63.4ms
Speed: 1.0ms preprocess, 63.4ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)


In [ ]:
speed = np.array([])
size = np.array([])
conf = np.array([])

for result in results:
    if len(result.boxes) == 0:
        continue

    speed = np.append(speed, [result.speed['inference']])

    bbox_dim = result.boxes.xywhn
    size = np.append(size, [bbox_dim[0][2] * bbox_dim[0][3]])
    
    conf = np.append(conf, [result.boxes.conf.numpy()[0]])

In [ ]:
xlim = [0, 0.05]
ylim = [90, 130]

plt.hist2d(size, speed, bins=50, cmap=plt.cm.Blues, range=[xlim, ylim])

plt.xlabel("Bounding Box Size ($pixels^{2}$)")
plt.ylabel("Inference Speed ($ms$)")
plt.xlim(xlim)
plt.ylim(ylim)
plt.colorbar()
plt.show()

In [ ]:
xlim = [0, 1.0]
ylim = [90, 130]

plt.hist2d(conf, speed, bins=50, cmap=plt.get_cmap('Blues'), range=[xlim, ylim])

plt.xlabel("Confidence")
plt.ylabel("Inference Speed ($ms$)")
plt.xlim(xlim)
plt.ylim(ylim)
plt.colorbar()
plt.show()

In [ ]:
bin_width = 1
min_bin = int(np.min(speed) - (np.min(speed) % bin_width))
max_bin = int(np.max(speed) + bin_width - (np.max(speed) % bin_width))

plt.hist(speed, bins=np.arange(min_bin, max_bin + bin_width, bin_width))

xlim = [int(min_bin - (min_bin % 10)), int(max_bin + 10 - (max_bin % 10))]

plt.xlabel("Inference Speed ($ms$)")
plt.ylabel("Frequency")
plt.xlim(xlim)
plt.show()

In [ ]:
bin_width = 0.05
min_bin = int(np.min(conf) - (np.min(conf) % bin_width))
max_bin = int(np.max(conf) + bin_width - (np.max(conf) % bin_width))

plt.hist(conf, bins=np.arange(min_bin, max_bin + bin_width, bin_width))

xlim = [0, 1]

plt.xlabel("Confidence")
plt.ylabel("Frequency")
plt.xlim(xlim)
plt.show()